In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import csv
import gzip

import scipy.io

import scipy.sparse as sps

from os.path import join
from sklearn.decomposition import PCA, IncrementalPCA

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

np.random.seed(1234)

sc.settings.verbosity = 3
sc.logging.print_header()

In [ ]:
adata_rna  = sc.read_h5ad('/home/yanxh/data/HumanFetal_50k/RNA/adata_rna_sampled.h5ad')
adata_atac = sc.read_h5ad('/home/yanxh/data/HumanFetal_50k/ATAC/adata_atac.h5ad')

# Integration using Portal

In [ ]:
import portal

# Specify the GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Create a folder for saving results
result_path = "./result"
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [ ]:
adata_rna.obs['domain'] = 'RNA'
adata_atac.obs['domain']= 'ATAC'

adata_rna.obs['cell_type'] = adata_rna.obs['Main_cluster_name'].values

In [ ]:
meta_rna  = adata_rna.obs.copy()
meta_atac = adata_atac.obs.copy() 

meta = pd.concat([meta_rna, meta_atac])
meta.shape

In [ ]:
## standard portal pipeline

model = portal.model.Model(training_steps=2000, 
                           lambdacos=10., lambdaAE=10., lambdaLA=10., lambdaGAN=1.0)
model.preprocess(adata_rna, adata_atac, hvg_num=4000, norm_pca=False) # perform preprocess and PCA
print('preprocessed feature dim: ', len(model.hvg_total))

model.train() # train the model
model.eval() # get integrated latent representation of cells

# portal.utils.plot_UMAP(model.latent, meta, colors=["domain", "cell_type"], save=False, result_path=result_path)